<a href="https://colab.research.google.com/github/BYUIDSS/DSS-ML-Bootcamp/blob/main/3b_Classification_Model/ML_Bootcamp_Classification_XGBoost_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Machine Learning Model 🧠  📈

---



### 🔴 PLEASE COPY THIS NOTEBOOK INTO YOUR OWN GITHUB OR GOOGLE DRIVE🔴
   File  ->  Save in Drive


## Overview

**XGBoost Classification is a supervised machine learning algorithm that builds an ensemble of decision trees to predict categorical outcomes. It is optimized for speed and performance using gradient boosting techniques.**

<br>

---

<br>

**Definition**  
XGBoost (Extreme Gradient Boosting) is an advanced implementation of gradient boosting designed for efficiency and accuracy. It improves predictions by sequentially training trees while correcting previous errors. The key components include:  

* **Boosting Trees**: A collection of decision trees built sequentially to reduce errors.  
* **Gradient Descent Optimization**: Adjusts model weights using the gradient of a loss function.  
* **Regularization**: Controls model complexity to prevent overfitting.  

For **classification**, XGBoost predicts categorical outcomes by minimizing a chosen loss function, such as logistic loss for binary classification or softmax (cross-entropy) loss for multi-class classification.

<br>

---

<br>

**Key Concepts**  
1. **Boosting Mechanism**:  
   - Unlike a single decision tree, XGBoost builds multiple trees in sequence.  
   - Each new tree corrects the errors of the previous ones by focusing on misclassified examples.

2. **Loss Functions**:  
   - Determines how errors are measured and minimized.  
   - Common choices for classification include:  
     * **Logistic Loss** – Used for binary classification tasks.  
     * **Softmax (Cross-Entropy Loss)** – Used for multi-class classification tasks.

3. **Regularization Techniques**:  
   - Prevents overfitting by adding penalties to complex models.  
   - **L1 Regularization (Lasso)** – Encourages sparsity by shrinking coefficients.  
   - **L2 Regularization (Ridge)** – Penalizes large coefficients to reduce variance.

4. **Feature Importance & Selection**:  
   - XGBoost ranks features by importance, aiding in feature selection.  
   - Helps in eliminating redundant or irrelevant features for better performance.

<br>

---

<br>

**Pros**  
1. **High Performance** – Optimized for speed, scalability, and efficiency.  
2. **Handles Missing Data** – Automatically learns how to deal with missing values.  
3. **Regularization Built-in** – Reduces overfitting with L1 and L2 penalties.  
4. **Probabilistic Predictions** – Provides probability scores for classification, enabling threshold tuning.  
5. **Works Well with Large Datasets** – Efficient memory usage and parallel processing.

<br>

---

<br>

**Cons**  
1. **Complexity** – More difficult to tune compared to simpler models.  
2. **Computationally Intensive** – Training can be slow on very large datasets.  
3. **Sensitive to Hyperparameters** – Performance depends on careful tuning of learning rate, tree depth, and regularization.  
4. **Less Interpretable** – Decision boundaries may be challenging to interpret compared to simpler models.

<br>

---

<br>

**Tips**  
* **Optimize Hyperparameters** – Use grid search or Bayesian optimization for tuning.  
* **Use Early Stopping** – Stop training if performance ceases to improve on validation data.  
* **Scale Features if Needed** – Although XGBoost can handle unscaled data, standardization might improve performance.  
* **Leverage Feature Importance** – Identify and remove less relevant features to improve efficiency.  
* **Adjust Decision Thresholds** – Fine-tune the probability threshold to balance precision and recall for your specific task.

<br>

---

<br>

**Useful Articles and Videos**  
* [XGBoost Official Documentation](https://xgboost.readthedocs.io/en/stable/index.html)  
* [XGBoost for Classification – Machine Learning Mastery](https://machinelearningmastery.com/xgboost-for-classification/)  
* [Understanding XGBoost – Analytics Vidhya](https://www.analyticsvidhya.com/blog/2016/01/xgboost-algorithm-easy-steps/)  
* [XGBoost Explained for Classification – YouTube](https://www.youtube.com/watch?v=8j0UDiN7my4)  

<br>


## Import Data/Libraries

In [6]:
!pip install lets_plot

In [12]:
# needed libraries for Classification models
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from collections import Counter

# Load the training dataset
#### If this gives an error go into the Data folder in GitHub and click on the data csv and then "Raw"
#### (underneath history in the upper righthand corner) then copy that url to replace the "data_raw_url"
data_raw_url = 'https://raw.githubusercontent.com/BYUIDSS/DSS-ML-Bootcamp/refs/heads/main/3b_Classification_Model/data/Churn_Modelling.csv?token=GHSAT0AAAAAAC6JBYNPUIMLJRDEKPFICXJQZ6XVWSQ'
banking_df = pd.read_csv(data_raw_url)

## Explore, Visualize and Understand the Data

In [13]:
banking_df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44.0,8,113755.78,2,1.0,0.0,149756.71,1
6,7,15592531,Bartlett,822,NaN,Male,50.0,7,0.00,2,1.0,1.0,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29.0,4,115046.74,4,1.0,0.0,119346.88,1
8,9,15792365,He,501,France,Male,44.0,4,142051.07,2,0.0,NaN,74940.50,0
9,10,15592389,H?,684,France,Male,NaN,2,134603.88,1,1.0,1.0,71725.73,0


## Feature Enginnering and Data Augmentation

### **Data Augmentation**  
**Definition:**
Data augmentation is the process of artificially expanding the size and diversity of a training dataset by applying transformations or modifications to the existing data while preserving the underlying labels or structure. It is commonly used in machine learning, especially in computer vision and natural language processing, to improve model performance and robustness.

### **Feature Engineering**  
**Definition:**
Feature engineering is the process of creating, modifying, or selecting relevant features (input variables) from raw data to improve the performance of a machine learning model. It involves transforming raw data into a format that makes it more suitable for algorithms to learn patterns.


## Machine Learning Model

### Split the data to train and test

### Create the model

### Train the model

### Make predictions

#### Hyperparameter Search

### Evaluate the Model

**Accuracy – The percentage of total predictions that are correct.**  
Example: If a spam filter correctly classifies 90 out of 100 emails (whether spam or not), the accuracy is 90%.  

**F1 Score  – Out of all the positive predictions, how many were actually correct.**  
Example: If a spam filter predicts 20 emails as spam, but only 15 are actually spam, precision is 15/20 = 75%.  


**Recall Score – Out of all the actual positive cases, how many did the model correctly identify.**  
Example: If there were 25 spam emails in total, and the model correctly identified 15 of them, recall is 15/25 = 60%.  

**Precision Score – A balance between precision and recall (harmonic mean).**  
Example: If precision is 75% and recall is 60%, F1 score is (2 × 75 × 60) / (75 + 60) = 66.7%.  



In [9]:
# Evaluate the model using classification metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared (R2)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)
print('Mean Absolute Error (MAE):', mae)
print('R-squared (R2):', r2)